In [1]:
import numpy as np
from bayes_opt import BayesianOptimization
from scipy import optimize

import sys
sys.path.append("../../lcls_cu_injector_ml_model/injector_surrogate/") 
sys.path.append("../../lcls_cu_injector_ml_model/configs/") 
sys.path.append("../../lcls_cu_injector_ml_model/models/") 
sys.path.append("../") 

# Suppress TF warnings
import tensorflow as tf
#tf.disable_v2_behavior()
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

from emittance_calc import get_normemit
from test_opt import TestOpt

Method w/ BO and Simplex:
1. ~~start w/ same init array for scan~~
2. ~~get beamsizes~~
3. ~~pass beamsizes + quad vals to adapt_range, get new range back~~
4. ~~get beamsizes for new range~~
5. ~~check symmetry~~
6. ~~get missing beamsizes if any~~
7. ~~check inflection points, get new sliced data back~~
8. ~~pass data to get_normemit~~

Extra step for BO:
1. ~~get emit + err returned from get_normemit~~
2. ~~if emit=nan or err/emit>0.15 -> go to next iteration~~

# taking bad emittances as large values / ignoring large uncertainty

In [2]:
seed = 21

bo_opt = TestOpt()
optimizer = bo_opt.run_bo_opt(rnd_state=seed, init_pnts=3, n_iter=25)


|   iter    |  target   |   varx    |   vary    |   varz    |
-------------------------------------------------------------
nemitx: 4.59 +/- 0.16 mm mrad
nemity: 8.73 +/- 1.03 mm mrad
|  1        | -6.33     |  0.4612   | -0.004218 |  0.004419 |
nemitx: 5.94 +/- 0.18 mm mrad
nemity: 10.60 +/- 0.47 mm mrad
|  2        | -7.936    |  0.4605   | -0.005882 | -0.008985 |
nemitx: 2.42 +/- 0.09 mm mrad
nemity: 2.36 +/- 0.52 mm mrad
|  3        | -2.387    |  0.4676   |  0.003278 | -0.003838 |
nemitx: 1.80 +/- 0.06 mm mrad
nemity: 1.08 +/- 0.03 mm mrad
|  4        | -1.393    |  0.4808   |  0.003899 | -0.006759 |
nemitx: 3.84 +/- 0.11 mm mrad
nemity: 4.95 +/- 1.08 mm mrad
|  5        | -4.36     |  0.4643   |  0.000915 |  0.006015 |
nemitx: 1.20 +/- 0.02 mm mrad
nemity: 1.50 +/- 0.01 mm mrad
|  6        | -1.343    |  0.4784   |  0.01     |  0.004021 |
nemitx: 3.60 +/- 0.15 mm mrad
nemity: 2.13 +/- 0.06 mm mrad
|  7        | -2.77     |  0.482    | -0.004233 |  0.004591 |
nemitx: 0.84 +/- 0.01

# removing bad emittances

In [2]:
seed = 21

bo_opt = TestOpt()
optimizer = bo_opt.run_bo_opt_w_reject(rnd_state=seed, init_pnts=3, n_iter=25)

nemitx: 2.85 +/- 0.05 mm mrad
nemity: 2.08 +/- 0.03 mm mrad
iter 0 | target 2.433 | config 0.481740 0.009279 0.009021
nemitx: 0.88 +/- 0.01 mm mrad
nemity: 0.98 +/- 0.10 mm mrad
iter 1 | target 0.929 | config 0.472294 0.005388 -0.002065
nemitx: 4.25 +/- 0.13 mm mrad
nemity: 6.64 +/- 1.08 mm mrad
iter 2 | target 5.314 | config 0.462690 -0.002541 0.003276
nemitx: 1.80 +/- 0.06 mm mrad
nemity: 1.08 +/- 0.03 mm mrad
iter 0 | target 1.393 | config 0.480793 0.003899 -0.006759
nemitx: 3.84 +/- 0.11 mm mrad
nemity: 4.95 +/- 1.08 mm mrad
iter 1 | target 4.360 | config 0.464307 0.000915 0.006015
nemitx: 1.03 +/- 0.02 mm mrad
nemity: 1.26 +/- 0.11 mm mrad
iter 2 | target 1.139 | config 0.471812 0.010000 -0.010000
nemitx: 3.60 +/- 0.15 mm mrad
nemity: 2.13 +/- 0.06 mm mrad
iter 3 | target 2.770 | config 0.482003 -0.004233 0.004591
nemitx: 7.93 +/- 0.24 mm mrad
nemity: 3.14 +/- 0.16 mm mrad
iter 4 | target 4.994 | config 0.485000 -0.010000 -0.010000
nemitx: 1.12 +/- 0.01 mm mrad
nemity: 1.27 +/- 0.

# Simplex

In [3]:
# take initial guess as best init of BO 
initial_guess = np.array([0.4676,  0.003278, -0.003838])

opt = TestOpt()
minimum = opt.run_simplex_opt(120, initial_guess) 

nemitx: 2.40 +/- 0.09 mm mrad
nemity: 2.33 +/- 0.52 mm mrad
nemitx: 14.47 +/- 1.86 mm mrad
nemity: 6.74 +/- 0.67 mm mrad
nemitx: 2.40 +/- 0.09 mm mrad
nemity: 2.33 +/- 0.52 mm mrad
nemitx: 2.40 +/- 0.09 mm mrad
nemity: 2.36 +/- 0.51 mm mrad
nemitx: 7.46 +/- 1.55 mm mrad
nemity: 31.48 +/- 1.60 mm mrad
nemitx: 2.06 +/- 0.08 mm mrad
nemity: 1.23 +/- 0.03 mm mrad
nemitx: 0.92 +/- 0.01 mm mrad
nemity: 1.04 +/- 0.01 mm mrad
nemitx: 1.41 +/- 0.03 mm mrad
nemity: 1.10 +/- 0.01 mm mrad
nemitx: 3.11 +/- 0.15 mm mrad
nemity: 1.53 +/- 0.07 mm mrad
Less than 3 data points were passed.
nan 3.397378782568114e-08
nemitx: 0.81 +/- 0.01 mm mrad
nemity: 1.00 +/- 0.06 mm mrad
nemitx: 1.03 +/- 0.01 mm mrad
nemity: 1.08 +/- 0.16 mm mrad
nemitx: 3.24 +/- 0.12 mm mrad
nemity: 3.71 +/- 0.65 mm mrad
nemitx: 1.07 +/- 0.03 mm mrad
nemity: 1.07 +/- 0.01 mm mrad
nemitx: 1.70 +/- 0.03 mm mrad
nemity: 1.57 +/- 0.35 mm mrad
nemitx: 0.88 +/- 0.00 mm mrad
nemity: 1.01 +/- 0.01 mm mrad
nemitx: 1.30 +/- 0.03 mm mrad
nemit

In [4]:
minimum

       allvecs: [array([ 0.4676  ,  0.003278, -0.003838]), array([ 0.48123833,  0.00330076, -0.00386465]), array([ 0.47669222,  0.00340244, -0.00366387]), array([ 0.47669222,  0.00340244, -0.00366387]), array([ 0.4728617 ,  0.00328931, -0.00380386]), array([ 0.4728617 ,  0.00328931, -0.00380386]), array([ 0.4728617 ,  0.00328931, -0.00380386]), array([ 0.4728617 ,  0.00328931, -0.00380386]), array([ 0.4728617 ,  0.00328931, -0.00380386]), array([ 0.47555107,  0.00338058, -0.00371922]), array([ 0.47515992,  0.00335571, -0.00378575]), array([ 0.47515992,  0.00335571, -0.00378575]), array([ 0.47515992,  0.00335571, -0.00378575]), array([ 0.47405691,  0.00336893, -0.00377413]), array([ 0.47405691,  0.00336893, -0.00377413]), array([ 0.47405691,  0.00336893, -0.00377413]), array([ 0.47440122,  0.00335613, -0.00378393]), array([ 0.47444156,  0.0033654 , -0.00375637]), array([ 0.47444156,  0.0033654 , -0.00375637]), array([ 0.47460064,  0.00337786, -0.00376513]), array([ 0.47460064,  0.003377